# GoogleNet ( Inception 3 )

In [2]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, SGD

## 参数定义

In [79]:

class image:
    def __init__(self):
        self.num_classes = 20

image_data = image()
image_data.num_classes

input_shape = (224, 224, 3)




## Inception layer

In [38]:
# create Inception model
# x is data input channel 
# filter_1 is the kernel with size 1*1,
# filter_3 is the kernel with size  1*1->3*3(tuple)
# filter_5 is the kernel with size 1*1->5*5 (tuple)
# filter_pool is the kernel with maxing pool -> 1*1

def inception(x, filter_1, filter_3, filter_5, filter_pool):
    #path1: 1*1 
    path1 = Conv2D(filters = filter_1, kernel_size = (1,1),strides=1,padding ='same',activation = 'relu')(x)
    #path2: 1*1->3*3
    path2 = Conv2D(filters = filter_3[0], kernel_size = (1,1),strides = 1,padding ='same',activation = 'relu')(x)
    path2 = Conv2D(filters = filter_3[1],kernel_size = (3,3),strides = 1,padding = 'same',activation = 'relu')(path2)
    #path3: 1*1 -> 5*5
    path3 = Conv2D(filters = filter_5[0], kernel_size = (1,1),strides = 1,padding ='same',activation = 'relu')(x)
    path3 = Conv2D(filters = filter_5[1],kernel_size=(5,5),strides = 1,padding = 'same',activation = 'relu')(path3)
    #path4: 3*3 -> 1*1
    path4 = MaxPooling2D(pool_size=(3,3),strides=1,padding='same')(x)
    path4 = Conv2D(filters=filter_pool,kernel_size=(1,1),strides=1,padding='same',activation='relu')(path4)
    
    return Concatenate(axis=-1)([path1,path2,path3,path4])
    #concatenate可以把tensor在对应的轴上的维度进行累加。-1表示最后一个轴(深度)
    

In [30]:
inputs = Input(shape=(224,224,3))
outputs = inception(inputs,filter_1=64,filter_3=(96,128),filter_5=(16,32),filter_pool=32)
model = Model(inputs = inputs, outputs = outputs)
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_49 (Conv2D)              (None, 224, 224, 96) 384         input_11[0][0]                   
__________________________________________________________________________________________________
conv2d_51 (Conv2D)              (None, 224, 224, 16) 64          input_11[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_7 (MaxPooling2D)  (None, 224, 224, 3)  0           input_11[0][0]                   
____________________________________________________________________________________________

## auxiliary
结构固定的一个辅助分类器

功能：

为了避免梯度消失，网络额外增加了2个辅助的softmax用于向前传导梯度（辅助分类器）。辅助分类器是将中间某一层的输出用作分类，并按一个较小的权重（0.3）加到最终分类结果中，这样相当于做了模型融合，同时给网络增加了反向传播的梯度信号，也提供了额外的正则化，对于整个网络的训练很有裨益。而在实际测试的时候，这两个额外的softmax会被去掉。

In [73]:
def auxiliary(x,name = None):
    layer = AveragePooling2D(pool_size=(5,5),strides=3,padding='valid')(x)
    layer = Conv2D(filters = 128,kernel_size=(1,1),strides=1,padding='same',activation='relu')(layer)
    layer = Flatten()(layer)
    layer = Dense(units=256,activation='relu')(layer)
    layer = Dropout(0.4)(layer)
    layer = Dense(units=image_data.num_classes,activation='softmax',name = name)(layer)
    return layer
    
    

In [75]:
inputs = Input(shape=(224,224,3))
outputs = auxiliary(inputs,name="Test")
model = Model(inputs = inputs, outputs = outputs)
model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_18 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
average_pooling2d_6 (Average (None, 74, 74, 3)         0         
_________________________________________________________________
conv2d_59 (Conv2D)           (None, 74, 74, 128)       512       
_________________________________________________________________
flatten_5 (Flatten)          (None, 700928)            0         
_________________________________________________________________
dense_11 (Dense)             (None, 256)               179437824 
_________________________________________________________________
dropout_5 (Dropout)          (None, 256)               0         
_________________________________________________________________
Test (Dense)                 (None, 20)                5140

In [100]:
def googlenet():
    
    #input layer
    inputs = Input(shape = input_shape)
    
    #stage 1: conv + maxpool + normalization
    stage1 = Conv2D(64,kernel_size=(7,7),strides = 2,padding='same',activation='relu')(inputs)
    stage1 = MaxPooling2D(pool_size=(3,3),strides=2,padding='same')(stage1)
    stage1 = BatchNormalization()(stage1)
    
    #stage 2: conv + conv + normalization + maxpooling
    stage2 = Conv2D(64,kernel_size=(1,1),strides=1,padding='same',activation='relu')(stage1)
    stage2 = Conv2D(192,kernel_size=(3,3),strides=1,padding='same',activation='relu')(stage2)
    stage2 = BatchNormalization()(stage2)
    stage2 = MaxPooling2D(pool_size=(3,3),strides=2,padding='same')(stage2)
    
    #stage-3 inception+inception+maxpooling
    
    stage3 = inception(stage2, 64,  (96,128), (16,32), 32)#3a
    stage3 = inception(stage3,128, (128,192), (32,96), 64)#3b
    stage3 = MaxPooling2D(pool_size=(3,3),strides=2,padding='same')(stage3)
    
    #stage-4 inception + auxiliary(from 4a) + inception*3 + auxiliary(from 4d) + inception+maxpool
    stage4_a = inception(stage3, 192,  (96,208),  (16,48),  64)#4a
    aux1  = auxiliary(stage4_a, name='aux1')#from stage4_a
    stage4_b = inception(stage4_a,160, (112,224),  (24,64),  64)#4b
    stage4_c = inception(stage4_b, 128, (128,256),  (24,64), 64) #4c
    stage4_d = inception(stage4_c, 112, (144,288),  (32,64),  64) #4d
    aux2  = auxiliary(stage4_d, name='aux2')#from stage4_d
    stage4_e = inception(stage4_d, 256, (160,320), (32,128), 128) #4e
    stage4 = MaxPooling2D(pool_size=(3,3),strides=2,padding='same')(stage4_e)
    
    #stage-5 inception*2+averagepool
    stage5 = inception(stage4, 256, (160,320), (32,128), 128) #5a
    stage5 = inception(stage5, 384, (192,384), (48,128), 128) #5b
    stage5 = AveragePooling2D(pool_size=(7,7), strides=1, padding='valid')(stage5)
    
    #stage -6 flatten+dense
    stage6 = Flatten()(stage5)
    stage6 = Dropout(0.4)(stage6)
    stage6 = Dense(units=256,activation='linear')(stage6)
    main = Dense(units=image_data.num_classes,activation='softmax',name='main')(stage6)
    
   
    model = Model(inputs= inputs,outputs=[main,aux1,aux2])
    return model
    
    

In [101]:
model=googlenet()
model.summary()

Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_29 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_389 (Conv2D)             (None, 112, 112, 64) 9472        input_29[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_86 (MaxPooling2D) (None, 56, 56, 64)   0           conv2d_389[0][0]                 
__________________________________________________________________________________________________
batch_normalization_17 (BatchNo (None, 56, 56, 64)   256         max_pooling2d_86[0][0]           
___________________________________________________________________________________________